# **Chapter 7. Secular Perturbations**

## **Action by Daniel Niño-Villegas, University of Antioquia**

In this notebook we will present part of the theoretical background of the chapter devoted to *Secular Perturbations* and some mathematical and numerical results which are interested for the theory.

For details on the theory please refer directly to the book:

> Murray, C. D., & Dermott, S. F. (1999). Solar system dynamics. Cambridge university press.

## Preliminaries

### Prerrequisites

In [ ]:
#!pip install -q rebound
#!pip install -q Fraction
#!pip install -q ipywidgets

### Other libraries

In [ ]:
#Global packages
import numpy as np
import matplotlib.pyplot as plt
import rebound as rb
import celluloid as cell

#Specific modules and routines
from tqdm import tqdm
from scipy.integrate import quad
from scipy.signal import savgol_filter
from ipywidgets import interact, widgets, fixed
from fractions import Fraction
from IPython.core.display import HTML

### Useful constants

In [ ]:
deg = np.pi/180
rad = 1/deg

### Plots aesthetics

In [ ]:
%matplotlib nbagg
#If you run this in Colab use
#%matplotlib inline

plt.rcParams['text.usetex'] = True
#If you don't have installed latex
#font for matplotlib, set this parameter
#to false. If you run this in Colab, set
#this parameter to false.

## **Section 7.2 Secular Perturbations for Two Planets**

### Experiment: Laplace coefficients study (7.1)

In the previous section we introduced the computation of the Laplace coefficients and their derivatives (up to second order), who are completely essential in the development of perturbation theory. This functions are until now completely "dark" in a sense that we don't know how they behave or even how their derivatives behave. So, we´ll now show an interactive plot of $b_{s}^{(j)}(\alpha)$, $D \, b_{s}^{(j)}(\alpha)$ and $D^2 \, b_{s}^{(j)}(\alpha)$ where you can modify the parameters $s$ and $j$ to see how the coefficents change:

In [ ]:
def blap_plot(s,j):
    a = np.linspace(0, 0.9, 1000)
    b = np.array([blap(x, s, j) for x in a])
    db = np.array([blap_dot(x, s, j) for x in a])
    ddb = np.array([blap_ddot(x, s, j) for x in a])
    
    j = int(j)
    s = str(Fraction(s).numerator) + '/' + str(Fraction(s).denominator)
    ss = fr'$s = {s}$' + r'$\hspace{20 pt}$' fr'$j = {j}$'
    
    plt.close()
    fig, axs = plt.subplots(3, 1, figsize=(4.5,7), sharex=True, dpi=110)
    axs[0].set_title(ss, fontsize=15)
    axs[0].set_ylabel(r'$b_{s}^{(j)}(\alpha)$', fontsize=17)
    axs[0].plot(a, b, 'k-')
    axs[1].set_xlabel(r'$\alpha$', fontsize=15)
    axs[1].set_ylabel(r'$D \, b_{s}^{(j)}(\alpha)$', fontsize=15)
    axs[1].plot(a, db, 'k-')
    axs[2].set_xlabel(r'$\alpha$', fontsize=15)
    axs[2].set_ylabel(r'$D^2 \, b_{s}^{(j)}(\alpha)$', fontsize=15)
    axs[2].plot(a, ddb, 'k-')
    fig.tight_layout()
    plt.subplots_adjust(hspace=0)
    plt.show()

In [ ]:
opciones = dict(continuous_update=False)
interact(blap_plot,
         s = widgets.FloatSlider(min=0.5, max=5, step=0.5, value=0.5, **opciones),
         j = widgets.FloatSlider(min=0, max=5, step=1, value=0, **opciones))

An important result of the previous plots is that the Laplace coefficients diverge rapidly near $\alpha = 1$, and so their derivatives. You can also check that if you increase the value of the $s$ parameter, the coefficient is virtually cero for a large number of $\alpha$ values, and diverge much quickly near $\alpha=1$. Be free to combine dfferent parameter values to check all of this for yourself.

### Experiment: excentricity and inclination vectors (7.2)

Initial conditions:

In [ ]:
#Partticle #1
a1 = 2
e1 = 0.02
pomega1 = 10*deg

#Particle #2
a2 = 3
e2 = 0.08
pomega2 = 250*deg

#Smulation t=0
sim = rb.Simulation()
sim.units = ('au', 'msun', 'yr')
sim.add(m=1, hash='Sun')
sim.add(m=1e-4, a=a1, e=e1, pomega=pomega1, hash='Particle #1')
sim.add(m=1e-4, a=a2, e=e2, pomega=pomega2, hash='Particle #2')
#sim.save('tmp/system.bin')

Orbit plot:

In [ ]:
#Orbits plot
fig, ax = rb.OrbitPlot(sim, unitlabel='[AU]', orbit_type='solid', lw=1.5, color=True)
fig.set_dpi(100)
fig.tight_layout()
plt.show()

Orbit periods:

In [ ]:
#Orbital periods
P1 = sim.particles['Particle #1'].P
n1 = 2*np.pi/P1
P2 = sim.particles['Particle #2'].P
n2 = 2*np.pi/P2

#Gravitational constant
G = sim.G

Integration parameters:

In [ ]:
#Integration parameters
sim.dt = P1/100
Nt = 1000
ts = np.linspace(0, 1000*P2, Nt)

Integration:

In [ ]:
#Integration
Es = np.zeros((Nt,4))
for i,t in enumerate(tqdm(ts, position=0, leave=True)):
    sim.integrate(t)
    sim.move_to_hel()
    orbits = sim.calculate_orbits()
    Es[i] = [orbits[0].e,
             np.mod(orbits[0].pomega,2*np.pi),
             orbits[1].e,
             np.mod(orbits[1].pomega,2*np.pi)]

In [ ]:
def vec_plot(e1, e2, pomega1, pomega2, i, s=1.1):
    i = int(i)
    
    hvec = np.zeros(2)
    kvec = np.zeros(2)
    
    hvec[0] = e1[i]*np.sin(pomega1[i])
    hvec[1] = e2[i]*np.sin(pomega2[i])
    
    kvec[0] = e1[i]*np.cos(pomega1[i])
    kvec[1] = e2[i]*np.cos(pomega2[i])
    
    h = np.sqrt(hvec.dot(hvec))
    k = np.sqrt(kvec.dot(kvec))
    L = max([h,k])
    
    hvec = np.array([hvec])/h
    kvec = np.array([kvec])/k
    
    origin = np.array([[0,0],[0,0]])
    
    plt.close()
    fig = plt.figure(dpi=110, figsize=(6,6))
    plt.xlim(-s*L,s*L)
    plt.ylim(-s*L,s*L)
    plt.plot([-s*L,s*L],[0,0], 'k-')
    plt.plot([0,0], [-s*L,s*L], 'k-')
    plt.quiver(*origin, hvec[:,0], hvec[:,1], scale_units='x', scale=1/h, color='k', label=r'$\vec{h}$')
    plt.quiver(*origin, kvec[:,0], kvec[:,1], scale_units='x', scale=1/k, color='r', label=r'$\vec{k}$')
    plt.legend(fontsize=14, loc=1)
    plt.show()
    
opciones = dict(continuous_update=False)
interact(vec_plot,
         e1 = fixed(Es[:,0]),
         e2 = fixed(Es[:,2]),
         pomega1 = fixed(Es[:,1]),
         pomega2 = fixed(Es[:,3]),
         i = widgets.FloatSlider(min=0, max=Nt-1, step=1, value=0, **opciones),
         s = fixed(1.1))

In [ ]:
plt.ioff()
fig, ax = plt.subplots(dpi=110)
camera = cell.Camera(fig)

n = 2

for e1, pomega1, e2, pomega2 in tqdm(zip(Es[::n,0], Es[::n,1], Es[::n,2], Es[::n,3])):
    vec_plot(e1, e2, pomega1, pomega2)
    camera.snap()
    
plt.close()
plt.ion();

In [ ]:
anim = camera.animate(interval=100)
anim.save("figs/orbit-animation.mp4")